In [63]:
from homeharvest import scrape_property
from thefuzz import fuzz
from thefuzz import process
import pandas as pd
import os


In [64]:
df = pd.read_csv(os.path.join('processed/NY_SLI_YEARS_ADJUSTED_NO_UNKNOWNS.csv'))

In [65]:
# get towns with the most lead
locality_counts = df[df['SL Category Cleaned'] == 'Lead']['Service Line Locality'].value_counts()


In [66]:
locality_counts


Service Line Locality
Elmira                       316
Village of Solvay            205
NEW ROCHELLE                 160
ROCHESTER                     77
Scarsdale                     76
Hudson                        74
Village of East Syracuse      72
EASTCHESTER                   48
Village of Elmira Heights     47
TUCKAHOE                      44
PELHAM                        41
PEL MANOR                     41
Rome                          40
BRONXVILLE                    36
Waterford                     34
PEARL RIVER                   24
SPRING VALLEY                 21
Elmria                        20
Onondaga                      19
Cayuga Heights                17
Geddes                        14
NANUET                        13
PIERMONT                      10
Glens Falls                    9
SPARKILL                       8
Salina                         7
Horseheads                     6
RYE                            6
PORT CHESTER                   5
Name: count, dtype: i

In [133]:
rows = []
from threading import Thread
last_data = None

for town, count in locality_counts.items():
    town_sa = df[df['Service Line Locality'] == town]
    lead_sa = town_sa[town_sa['SL Category Cleaned'] == 'Lead']
    random_sa = None
    
    if len(lead_sa) == 0:
        print(f'No lead in {town}')
        random_sa = town_sa.sample(n=1)
    else:
        print(f'{len(lead_sa)} lead pipes in {town}')
        random_sa = lead_sa.sample(n=min(1, len(lead_sa)))


    for i in range(len(random_sa)):
        center = str(random_sa['Street Address'].values[i])
        print(center)
        limit = 10000
        if count < 30:
            limit = count
        data = scrape_property(location=center + ', ' + town + ', NY', 
                        return_type='pandas',
                        radius=10,
                        limit=limit
        )
        if len(data) == 0:
            print('no data with given street address')
            data = scrape_property(location=town + ', NY', 
                        return_type='pandas',
                        limit=limit
            )

        print(f'got {len(data)} rows for {town}')
        this_town = df[df['Service Line Locality'] == town]

        def process(data, this_town, town):
            rows_to_keep = []
            data['street'] = data['street'].fillna('')
            this_town['Street Address'] = this_town['Street Address'].fillna('')
            this_town['street'] = this_town['Street Address'].apply(lambda x: x.upper().replace('.', ''))
            this_town['city'] = this_town['Service Line Locality'].apply(lambda x: x.upper().replace('.', ''))

            for i, row in data.iterrows():
                x = row['street'].upper().replace('.', '')
                if any(x == s for s in this_town['street']):
                    rows_to_keep.append(i)

            keep = data.loc[rows_to_keep]   
            print(f'keeping {len(keep)} rows of {town}')
            
            keep['street'] = keep['street'].apply(lambda x: x.upper().replace('.', ''))
            keep['city'] = keep['city'].apply(lambda x: x.upper().replace('.', ''))

            merged = pd.merge(keep, this_town, on=['city', 'street'])
            print(len(merged))
            rows.append(merged)
            
        t = Thread(target=process, args=(data, this_town, town))
        t.start()


316 lead pipes in Elmira
401 MADISON AVE
got 317 rows for Elmira
205 lead pipes in Village of Solvay
117 RUSSET LN


/tmp/ipykernel_77896/2392718012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['Street Address'] = this_town['Street Address'].fillna('')
/tmp/ipykernel_77896/2392718012.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['street'] = this_town['Street Address'].apply(lambda x: x.upper().replace('.', ''))
/tmp/ipykernel_77896/2392718012.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

keeping 93 rows of Elmira
93
got 21 rows for Village of Solvay
keeping 7 rows of Village of Solvay
0
160 lead pipes in NEW ROCHELLE
21 RISLEY PL


/tmp/ipykernel_77896/2392718012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['Street Address'] = this_town['Street Address'].fillna('')
/tmp/ipykernel_77896/2392718012.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['street'] = this_town['Street Address'].apply(lambda x: x.upper().replace('.', ''))
/tmp/ipykernel_77896/2392718012.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

got 6531 rows for NEW ROCHELLE
77 lead pipes in ROCHESTER
144 Oneida St


...

/tmp/ipykernel_77896/2392718012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['Street Address'] = this_town['Street Address'].fillna('')
/tmp/ipykernel_77896/2392718012.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['street'] = this_town['Street Address'].apply(lambda x: x.upper().replace('.', ''))
/tmp/ipykernel_77896/2392718012.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

got 5458 rows for TUCKAHOE
41 lead pipes in PELHAM
360 7TH AVE


/tmp/ipykernel_77896/2392718012.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['Street Address'] = this_town['Street Address'].fillna('')
/tmp/ipykernel_77896/2392718012.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_town['street'] = this_town['Street Address'].apply(lambda x: x.upper().replace('.', ''))
/tmp/ipykernel_77896/2392718012.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

keeping 3 rows of TUCKAHOE
2


KeyboardInterrupt: 

In [134]:
len(rows)
output = pd.concat(rows)
output = output.drop_duplicates(subset=['street', 'city'])
print(len(output))

output.to_csv('processed/NY_SLI_SCRAPED.csv')


105


In [131]:
output = output.drop_duplicates(subset=['street', 'city'])
len(output)

100

In [132]:
output['SL Category Cleaned'].value_counts()

SL Category Cleaned
Non-Lead    98
Lead         2
Name: count, dtype: int64